In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os
import xml.etree.ElementTree as ET

def xml_to_txt(xml_path, txt_path, class_map):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    with open(txt_path, 'w') as txt_file:
        for obj in root.findall('object'):
            class_name = obj.find('name').text
            if class_name not in class_map:
                continue
            class_index = class_map[class_name]
            bbox = obj.find('bndbox')
            xmin = int(bbox.find('xmin').text)
            ymin = int(bbox.find('ymin').text)
            xmax = int(bbox.find('xmax').text)
            ymax = int(bbox.find('ymax').text)

            # Calculate normalized coordinates
            img_width = int(root.find('size/width').text)
            img_height = int(root.find('size/height').text)
            x_center = (xmin + xmax) / 2 / img_width
            y_center = (ymin + ymax) / 2 / img_height
            width = (xmax - xmin) / img_width
            height = (ymax - ymin) / img_height

            # Write to txt file in Ultralytics format
            txt_file.write(f"{class_index} {x_center} {y_center} {width} {height}\n")

def batch_convert(xml_folder, txt_folder, class_map):
    os.makedirs(txt_folder, exist_ok=True)
    for xml_file in os.listdir(xml_folder):
        if xml_file.endswith('.xml'):
            xml_path = os.path.join(xml_folder, xml_file)
            txt_file = os.path.join(txt_folder, xml_file.replace('.xml', '.txt'))
            xml_to_txt(xml_path, txt_file, class_map)

# Class mapping (class name to index)
class_map = {
    'Business Passenger Car': 0,
    'Private Passenger Car': 1,
    'Bus': 2,
    'Motorcycle': 3,
    # Add more classes as needed
}

# 使用示例
xml_folder = '/content/gdrive/MyDrive/ComputerVisionDeveloper/TrainYolov8CustomDataset/datasets/coco/labels/val-xml'
txt_folder = '/content/gdrive/MyDrive/ComputerVisionDeveloper/TrainYolov8CustomDataset/datasets/coco/labels/val'

batch_convert(xml_folder, txt_folder, class_map)